# Create plaquette using the RPNG format

## 1. RPNG format

```
-z1- -z2- -z3- -z4-
rpng rpng rpng rpng
```

(r) data qubit reset basis or -  
(p) data basis for the controlled operation (x means CNOT controlled on the ancilla and targeting the data qubit, y means CZ)  
(n) time step  
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- the ancilla is always initialized in $\ket{+}$ and measured in the X basis
- the ancilla is always the control qubit for the CNOT and CZ gates
- time step of r same as ancilla reset (default 0)
- time step of g same as ancilla measurement (default 6)

## 2. Extended RPNG format

```
z0z5 -xz1- -xz2- -xz3- -xz4-
pnpn rppng rppng rppng rppng
```

(p) ancilla init basis  
(n) time step  
(p) ancilla measure basis  
(n) time step

(r) data qubit reset basis or h or -  
(pp) ancilla-data 2-qubit bases (xz means CNOT targeting the ancilla)  
(n) time step  
(g) data qubit measure basis or h or -

Assumptions on the circuit:
- if not otherwise stated, a basis can be {x,y,z}
- at least one of the (pp) must be z, indicating the control qubit
- time step of r same as ancilla reset
- time step of g same as ancilla measurement

## 3. Test the functionality

In [1]:
from typing import Literal
from pathlib import Path
import stim
import matplotlib.pyplot as plt

from tqec import (
    Plaquette,
    PlaquetteQubits,
    QubitMap,
    ScheduledCircuit,
    SquarePlaquetteQubits,
    validate_rpng_string,
    create_plaquette_from_rpng_string,
)

ASSETS_FOLDER = Path("../../assets/").resolve()

In [2]:
rpng = '-x5h -z2z -x3x hz1-' # correct
qubits = SquarePlaquetteQubits()

format = validate_rpng_string(rpng)
if format == 1:
    print('-> RPNG is in simplified format\n')
elif format == 2:
    print('-> RPNG is in extended format\n')

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
except ValueError as err:
    print(f'{err}')

print('-> Circuit associated with the plaquette:\n')
print(plaq.circuit.get_circuit(include_qubit_coords=True))

-> RPNG is in simplified format

-> Circuit associated with the plaquette:

QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4


In [3]:
# Simplified RPNG format.
rpng = '---- ---- ---- ---- ----' # wrong number of values
rpng = '---- ---- --- ----' # wrong length of values
rpng = '---- ---- ---- ----' # correct but uneventful
rpng = '-z1- -z2- -z3- -z4-' # correct
rpng = '-z1- -z2- ---- -z4-' # wrong number of 2Q gates
rpng = '-z1- -z4- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z1- -z6- -z3- -z4-' # wrong times for the 2Q gates
rpng = '-z5- -x2- -x3- -z1-' # correct
rpng = '-x5h -z2z -x3x hz1-' # correct

# Extended RPNG format.
rpng = 'z0z5 -xz1- -xz2- -xz3- -xz4-' # wrong times for the 2Q gates
rpng = 'z0z3 -xz1- ----- -xz2- -----' # correct
rpng = 'z3z0 -xx1- ----- -xz2- ----' # wrong meas time
rpng = 'z0z5 -xz1- -xz2- -xz3- -xz4-' # correct

try:
    plaq = create_plaquette_from_rpng_string(rpng_string = rpng)
    print(f'Circuit associated with the plaquette "{rpng}":\n')
    print(plaq.circuit.get_circuit(include_qubit_coords=True))
except ValueError as err:
    print(f'{err}')

Circuit associated with the plaquette "z0z5 -xz1- -xz2- -xz3- -xz4-":

QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
RX 4
TICK
CX 0 4
TICK
CX 1 4
TICK
CX 2 4
TICK
CX 3 4
TICK
MX 4


## 4. Example: associate a custom circuit to the plaquette

**TODO:** Here we just copy the code from the tqec implementation.

In [4]:
PLAQUETTE_PREP_TIME = 0
PLAQUETTE_MEAS_TIME = 6

def custom_create_plaquette_from_rpng_string(
        rpng_string: str,
        qubits: PlaquetteQubits,
        prep_time: int = PLAQUETTE_PREP_TIME,
        meas_time: int = PLAQUETTE_MEAS_TIME
    ) -> Plaquette:
    """Create a plaquette from the RPNG format
    
    Assumptions:
    - the ancilla qubit is the last among the PlaquetteQubits (thus has index 4)
    """
    format = validate_rpng_string(rpng_string, prep_time=prep_time, meas_time=meas_time)
    if format == 1:
        print('simplified RPNG format')
        circuit_as_list = [''] * (meas_time - prep_time + 1)
        for q, v in enumerate(rpng_string.split(' ')):
            # 2Q gates.
            circuit_as_list[int(v[2])] += f'C{v[1].upper()} 4 {q}\n'
            # Data reset or Hadamard.
            if v[0] == 'h':
                circuit_as_list[0] += f'H {q}\n'
            elif v[0] != '-':
                circuit_as_list[0] += f'R{v[0].upper()} {q}\n'
            # Data measurement or Hadamard.
            if v[3] == 'h':
                circuit_as_list[-1] += f'H {q}\n'
            elif v[3] != '-':
                circuit_as_list[-1] += f'M{v[3].upper()} {q}\n'
        # Ancilla reset and measurement.
        circuit_as_list[0] += 'RX 4\n'
        circuit_as_list[-1] += 'MX 4\n'
    elif format == 2:
        print('extended RPNG format')
        # Update prep and meas times.
        values = rpng_string.split(' ')
        prep_time = int(values[0][1])
        meas_time = int(values[0][3])
        circuit_as_list = [''] * (meas_time - prep_time + 1)
        for q, v in enumerate(values[1:]):
            # 2Q gates.
            if v[1] == 'z':
                circuit_as_list[int(v[3])] += f'C{v[2].upper()} 4 {q}\n'
            elif v[2] == 'z':
                circuit_as_list[int(v[3])] += f'C{v[1].upper()} {q} 4\n'
            # Data reset or Hadamard.
            if v[0] == 'h':
                circuit_as_list[0] += f'H {q}\n'
            elif v[0] != '-':
                circuit_as_list[0] += f'R{v[0].upper()} {q}\n'
            # Data measurement or Hadamard.
            if v[4] == 'h':
                circuit_as_list[-1] += f'H {q}\n'
            elif v[4] != '-':
                circuit_as_list[-1] += f'M{v[4].upper()} {q}\n'
        # Ancilla reset and measurement.
        circuit_as_list[0] += 'RX 4\n'
        circuit_as_list[-1] += 'MX 4\n'
    elif format == 0:
        raise ValueError(f'invalide rpng string "{rpng_string}"')
    q_map = QubitMap.from_qubits(qubits)
    circuit_as_str = "TICK\n".join(circuit_as_list)
    circuit = stim.Circuit(circuit_as_str)
    scheduled_circuit = ScheduledCircuit.from_circuit(circuit, qubit_map = q_map)
    return Plaquette(name = 'test', qubits = qubits, circuit = scheduled_circuit)

rpng = '-x5h -z2z -x3x hz1-' # correct
qubits = SquarePlaquetteQubits()

try:
    plaq = custom_create_plaquette_from_rpng_string(rpng_string = rpng, qubits = qubits)
    print(plaq.circuit.get_circuit(include_qubit_coords=True))
except ValueError as err:
    print(f'{err}')

simplified RPNG format
QUBIT_COORDS(-1, -1) 0
QUBIT_COORDS(1, -1) 1
QUBIT_COORDS(-1, 1) 2
QUBIT_COORDS(1, 1) 3
QUBIT_COORDS(0, 0) 4
H 3
RX 4
TICK
CZ 4 3
TICK
CZ 4 1
TICK
CX 4 2
TICK
TICK
CX 4 0
TICK
H 0
M 1
MX 2 4
